# Build County FIPS Crosswalk from All-Geocodes File

This notebook processes the official Census **all-geocodes-v2017.xlsx** file into a county crosswalk lookup table.
The output will be a CSV file (`county_fips_crosswalk.csv`) with columns:
- fips (5-digit)
- state_name
- county_name


In [3]:
import pandas as pd

# --- 1. Load All-Geocodes file ---
all_geocodes_path = "all-geocodes-v2017.xlsx"  # update if needed

df_all = pd.read_excel(all_geocodes_path, skiprows=4, dtype=str)
df_all.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,010,00,000,00000,00000,00000,United States
1,040,01,000,00000,00000,00000,Alabama
2,050,01,001,00000,00000,00000,Autauga County
3,050,01,003,00000,00000,00000,Baldwin County
4,050,01,005,00000,00000,00000,Barbour County


In [5]:
# --- 2. Filter to counties only (Summary Level = 050) ---
df_counties = df_all[df_all["Summary Level"] == "050"].copy()

# --- 3. Build full FIPS code ---
df_counties["fips"] = df_counties["State Code (FIPS)"] + df_counties["County Code (FIPS)"]

# --- 4. Rename columns ---
df_counties = df_counties.rename(columns={
    "State Code (FIPS)": "state_fips",
    "County Code (FIPS)": "county_fips",
    "Area Name (including legal/statistical area description)": "county_name"
})
df_counties.head()

,Summary Level,state_fips,county_fips,County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),county_name,fips
2,050,01,001,00000,00000,00000,Autauga County,01001
3,050,01,003,00000,00000,00000,Baldwin County,01003
4,050,01,005,00000,00000,00000,Barbour County,01005
5,050,01,007,00000,00000,00000,Bibb County,01007
6,050,01,009,00000,00000,00000,Blount County,01009


In [6]:
# --- 5. Extract state names (Summary Level = 040) ---
df_states = df_all[df_all["Summary Level"] == "040"][
    ["State Code (FIPS)", "Area Name (including legal/statistical area description)"]
].copy()

df_states = df_states.rename(columns={
    "State Code (FIPS)": "state_fips",
    "Area Name (including legal/statistical area description)": "state_name"
})
df_states.head()

,state_fips,state_name
1,01,Alabama
530,02,Alaska
708,04,Arizona
815,05,Arkansas
1392,06,California


In [7]:
# --- 6. Merge counties with state names ---
df_crosswalk = df_counties.merge(df_states, on="state_fips", how="left")
df_crosswalk = df_crosswalk[["fips", "state_name", "county_name"]]
df_crosswalk.head()

,fips,state_name,county_name
0,01001,Alabama,Autauga County
1,01003,Alabama,Baldwin County
2,01005,Alabama,Barbour County
3,01007,Alabama,Bibb County
4,01009,Alabama,Blount County


In [8]:
# --- 7. Save to CSV ---
df_crosswalk.to_csv("county_fips_crosswalk.csv", index=False)
print("Crosswalk saved as county_fips_crosswalk.csv")

Crosswalk saved as county_fips_crosswalk.csv
